In [1]:
# https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/
import os
import ast
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torchvision
import torch.nn.functional as F
import torch
import torch.optim as optim
import helper
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data.sampler import SubsetRandomSampler
# import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from imblearn.under_sampling import RandomUnderSampler

num_classes = 12
learning_rate = 0.001
num_epochs = 2
batch_size = 64
patience = 1
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
ROOT_DIR = "/workspace/resnet/"
ROOT_DATA_DIR = "/workspace/resnet/data/raw"
ARCHIVE_DIR = os.path.join(ROOT_DATA_DIR, "archive.zip")
DATA_DIR = os.path.join(ROOT_DATA_DIR)#, "labelme-12-50k")
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TRAIN_ANNOT_PATH = os.path.join(TRAIN_DIR, "annotation.txt")
TEST_DIR = os.path.join(DATA_DIR, "test")
TEST_ANNOT_PATH = os.path.join(TEST_DIR, "annotation.txt")
CLASSES_TXT_PATH = os.path.join(DATA_DIR, "classes.txt")
INTERIM_DATA_DIR = "/workspace/resnet/data/interim/"
MODEL_DIR = os.path.join(ROOT_DIR, "models")
CNN_PATH = os.path.join(MODEL_DIR, "2.3_cnn-balanced-2epochs")

# import zipfile
# with zipfile.ZipFile(ARCHIVE_DIR, 'r') as zip_ref:
#     zip_ref.extractall(ROOT_DATA_DIR)

cpu


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
train_df_balanced = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"balanced-train-40000.csv"))
test_df = pd.read_csv(os.path.join(INTERIM_DATA_DIR,"test-10000.csv"))

In [4]:
transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
            )

In [5]:
class MyDataset(Dataset):
    def __init__(self,annot_df,transform = None):
        self.annotations = annot_df
        self.transform = transform
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        img = Image.open(self.annotations.img_path[index])
        y_label = torch.tensor(int(self.annotations.int_label[index]))
        if self.transform:
            image = transform(img)
        else:
            image = img
        return (image,y_label)

In [6]:
# train_data = MyDataset(train_df,transform = transform)
test_data = MyDataset(test_df,transform = transform)
train_data_balanced = MyDataset(train_df_balanced,transform = transform)


# train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
train_loader_balanced = DataLoader(dataset = train_data_balanced, batch_size = batch_size, shuffle = True)

test_loader

In [7]:
train_df_balanced.int_label.value_counts()

0     3438
9     3422
7     3392
6     3325
11    3317
10    3312
8     3305
1     3303
2     3302
4     3297
5     3295
3     3292
Name: int_label, dtype: int64

In [8]:
class CNN(nn.Module):
    def __init__(self,num_classes):
        super(CNN,self).__init__()
        # input channels is 3 because of RGB channels
        #((w-f+2P)/s) + 1
#         self.conv_l1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
#         self.conv_l2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        self.conv_l1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3)
        self.conv_l2 = nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = 3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_l3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
#         self.conv_l4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.conv_l3 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3)
        self.conv_l4 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_l5 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.conv_l6 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
        self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
#         self.conv_l5 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
#         self.conv_l6 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3)
#         self.max_pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(50176,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128,num_classes)
        
    def forward(self,x):
#         print(x.size())
        out = self.conv_l1(x)
#         print(out.size())
        out = self.conv_l2(out)
#         print(out.size())
        out = self.max_pool1(out)
#         print(out.size())
        out = self.conv_l3(out)
#         print(out.size())
        out = self.conv_l4(out)
#         print(out.size())
        out = self.max_pool2(out)
    
        out = self.conv_l5(out)
#         print(out.size())
        out = self.conv_l6(out)
#         print(out.size())
        out = self.max_pool2(out)
#         print(out.size())
        out = out.reshape(out.size(0), -1)
        
#         print(out.size())
        out = self.fc1(out)
#         print(out.size())
        out = self.relu1(out)
#         print(out.size())
        out = self.fc2(out)
        return out

# layers = [module for module in model.modules()][1:]
# summary(CNN(12),(32, 3, 256, 256))
# layers

model = CNN(num_classes)
print(model)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

# total_step = len(train_loader)
total_step = len(train_loader_balanced)

best_score = None
counter = 0

for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_loader_balanced):
#     for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
        
    if best_score is None:
        best_score = loss
    else:
            # Check if val_loss improves or not.
        if loss < best_score:
                # val_loss improves, we update the latest best_score, 
                # and save the current model
            best_score = loss
            torch.save(model.state_dict(), CNN_PATH)
        else:
                # val_loss does not improve, we increase the counter, 
                # stop training if it exceeds the amount of patience
            counter += 1
            if counter >= patience:
                break
    if epoch == num_epochs:
        torch.save(model.state_dict(), CNN_PATH)

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


CNN(
  (conv_l1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_l2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)
Epoch [1/2], Loss: 2.4812
Epoch [1/2], Loss: 2.4890
Epoch [1/2], Loss: 2.4837
Epoch [1/2], Loss: 2.4863
Epoch [1/2], Loss: 2.4872
Epoch [1/2], Loss: 2.4955
Epoch [1/2], Loss: 2.4837
Epoch [1/2], Loss: 2.

Epoch [1/2], Loss: 2.0125
Epoch [1/2], Loss: 2.0534
Epoch [1/2], Loss: 1.8138
Epoch [1/2], Loss: 2.1418
Epoch [1/2], Loss: 2.0031
Epoch [1/2], Loss: 2.1343
Epoch [1/2], Loss: 2.2274
Epoch [1/2], Loss: 1.9956
Epoch [1/2], Loss: 1.9587
Epoch [1/2], Loss: 1.9313
Epoch [1/2], Loss: 1.8295
Epoch [1/2], Loss: 2.1304
Epoch [1/2], Loss: 2.0833
Epoch [1/2], Loss: 2.1069
Epoch [1/2], Loss: 2.1773
Epoch [1/2], Loss: 1.7665
Epoch [1/2], Loss: 1.7528
Epoch [1/2], Loss: 2.0668
Epoch [1/2], Loss: 2.1677
Epoch [1/2], Loss: 1.9939
Epoch [1/2], Loss: 2.0310
Epoch [1/2], Loss: 1.8210
Epoch [1/2], Loss: 1.9768
Epoch [1/2], Loss: 1.7698
Epoch [1/2], Loss: 1.8926
Epoch [1/2], Loss: 1.9441
Epoch [1/2], Loss: 1.9589
Epoch [1/2], Loss: 1.7564
Epoch [1/2], Loss: 1.9165
Epoch [1/2], Loss: 2.0138
Epoch [1/2], Loss: 1.9949
Epoch [1/2], Loss: 1.9434
Epoch [1/2], Loss: 2.1154
Epoch [1/2], Loss: 1.7220
Epoch [1/2], Loss: 2.0170
Epoch [1/2], Loss: 1.9082
Epoch [1/2], Loss: 2.0663
Epoch [1/2], Loss: 2.1460
Epoch [1/2],

Epoch [1/2], Loss: 1.5406
Epoch [1/2], Loss: 1.6887
Epoch [1/2], Loss: 1.6429
Epoch [1/2], Loss: 1.2843
Epoch [1/2], Loss: 1.6536
Epoch [1/2], Loss: 1.4942
Epoch [1/2], Loss: 1.5599
Epoch [1/2], Loss: 1.3161
Epoch [1/2], Loss: 2.0185
Epoch [1/2], Loss: 1.6520
Epoch [1/2], Loss: 1.5060
Epoch [1/2], Loss: 1.6989
Epoch [1/2], Loss: 1.3928
Epoch [1/2], Loss: 1.5167
Epoch [1/2], Loss: 1.3822
Epoch [1/2], Loss: 1.3636
Epoch [1/2], Loss: 1.4093
Epoch [1/2], Loss: 1.2877
Epoch [1/2], Loss: 1.5283
Epoch [1/2], Loss: 1.5593
Epoch [1/2], Loss: 1.1961
Epoch [1/2], Loss: 1.6584
Epoch [1/2], Loss: 1.4067
Epoch [1/2], Loss: 1.3103
Epoch [1/2], Loss: 1.2550
Epoch [1/2], Loss: 1.2550
Epoch [2/2], Loss: 1.2650
Epoch [2/2], Loss: 1.3770
Epoch [2/2], Loss: 1.1319
Epoch [2/2], Loss: 1.5264
Epoch [2/2], Loss: 1.5841
Epoch [2/2], Loss: 1.5545
Epoch [2/2], Loss: 1.4779
Epoch [2/2], Loss: 1.7865
Epoch [2/2], Loss: 1.2644
Epoch [2/2], Loss: 1.4868
Epoch [2/2], Loss: 1.3027
Epoch [2/2], Loss: 1.6099
Epoch [2/2],

Epoch [2/2], Loss: 1.0525
Epoch [2/2], Loss: 1.0359
Epoch [2/2], Loss: 0.9554
Epoch [2/2], Loss: 0.8278
Epoch [2/2], Loss: 1.1552
Epoch [2/2], Loss: 1.0757
Epoch [2/2], Loss: 0.9713
Epoch [2/2], Loss: 0.9271
Epoch [2/2], Loss: 1.1664
Epoch [2/2], Loss: 1.1403
Epoch [2/2], Loss: 1.1706
Epoch [2/2], Loss: 1.0756
Epoch [2/2], Loss: 0.8560
Epoch [2/2], Loss: 0.8953
Epoch [2/2], Loss: 0.9503
Epoch [2/2], Loss: 1.0154
Epoch [2/2], Loss: 1.0288
Epoch [2/2], Loss: 1.0418
Epoch [2/2], Loss: 1.3232
Epoch [2/2], Loss: 1.0055
Epoch [2/2], Loss: 1.0351
Epoch [2/2], Loss: 1.1072
Epoch [2/2], Loss: 0.9696
Epoch [2/2], Loss: 0.9553
Epoch [2/2], Loss: 0.9879
Epoch [2/2], Loss: 1.3910
Epoch [2/2], Loss: 0.9905
Epoch [2/2], Loss: 0.9135
Epoch [2/2], Loss: 1.1097
Epoch [2/2], Loss: 1.0943
Epoch [2/2], Loss: 0.8785
Epoch [2/2], Loss: 0.9972
Epoch [2/2], Loss: 1.0391
Epoch [2/2], Loss: 1.0216
Epoch [2/2], Loss: 1.1403
Epoch [2/2], Loss: 1.0330
Epoch [2/2], Loss: 0.9070
Epoch [2/2], Loss: 0.7351
Epoch [2/2],

Epoch [2/2], Loss: 0.7309
Epoch [2/2], Loss: 0.6005
Epoch [2/2], Loss: 0.6408
Epoch [2/2], Loss: 0.9657
Epoch [2/2], Loss: 0.4903
Epoch [2/2], Loss: 0.8308
Epoch [2/2], Loss: 0.7741
Epoch [2/2], Loss: 0.9151
Epoch [2/2], Loss: 0.7666
Epoch [2/2], Loss: 0.5804
Epoch [2/2], Loss: 0.6437
Epoch [2/2], Loss: 0.7126
Epoch [2/2], Loss: 0.7399
Epoch [2/2], Loss: 0.5268
Epoch [2/2], Loss: 0.6451
Epoch [2/2], Loss: 0.7675
Epoch [2/2], Loss: 0.4846
Epoch [2/2], Loss: 0.3952
Epoch [2/2], Loss: 0.6587
Epoch [2/2], Loss: 0.6587


In [9]:
model.eval()

CNN(
  (conv_l1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv_l2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_l5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv_l6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (max_pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=12, bias=True)
)

In [10]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader_balanced:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 40000 train images: 80.9675 %


In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))




Accuracy of the network on the 10000 test images: 45.4 %


In [12]:
summary(CNN(12),(3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 254, 254]             896
            Conv2d-2         [-1, 32, 252, 252]           9,248
         MaxPool2d-3         [-1, 32, 126, 126]               0
            Conv2d-4         [-1, 64, 124, 124]          18,496
            Conv2d-5         [-1, 64, 122, 122]          36,928
         MaxPool2d-6           [-1, 64, 61, 61]               0
            Conv2d-7           [-1, 64, 59, 59]          36,928
            Conv2d-8           [-1, 64, 57, 57]          36,928
         MaxPool2d-9           [-1, 64, 28, 28]               0
           Linear-10                  [-1, 128]       6,422,656
             ReLU-11                  [-1, 128]               0
           Linear-12                   [-1, 12]           1,548
Total params: 6,563,628
Trainable params: 6,563,628
Non-trainable params: 0
---------------------------

In [13]:
images, labels = next(iter(train_loader))
images.shape

NameError: name 'train_loader' is not defined

In [ ]:
images[0].shape

In [ ]:
labels.shape

In [ ]:
torchvision.transforms.functional.get_image_size(images)

In [ ]:
print(test_df.str_label[1500])
Image.open(test_df.img_path[1500])
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
print(train_df.str_label[1000],train_df.img_path[1000])
Image.open(train_df.img_path[1000])

In [ ]:
# transform = transforms.Compose(
#         [transforms.ToTensor(),
#         transforms.Normalize((0.5,0.5,0.5),(0.5, 0.5, 0.5))]
#     )

# test_data = datasets.ImageFolder('/workspace/resnet/data/raw/test/', transform=transform)
# test_data_loader  = data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data = datasets.ImageFolder('/workspace/resnet/data/raw/train/', transform=transform)
# train_data_loader  = data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4) 

# train_data